In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# 06 - Tensor networks: MPS/MPO\n",
        "\n",
        "We construct random MPS objects and compare contraction inner products against dense reconstruction.\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import tensorflow as tf\n",
        "from tig.core.random import Rng\n",
        "from tig.tensor_networks.mps import MPS, mps_inner, mps_to_dense\n",
        "\n",
        "tf.keras.backend.set_floatx(\"float64\")\n",
        "tf.config.run_functions_eagerly(True)\n",
        "\n",
        "def random_mps(rng: Rng, length: int, phys_dim: int, bond_dim: int) -> MPS:\n",
        "    cores = []\n",
        "    r_prev = 1\n",
        "    for k in range(length):\n",
        "        r_next = 1 if k == length - 1 else bond_dim\n",
        "        cores.append(rng.normal((r_prev, phys_dim, r_next), dtype=tf.float64))\n",
        "        r_prev = r_next\n",
        "    return MPS(cores=cores)\n",
        "\n",
        "rng = Rng(seed=0)\n",
        "a = random_mps(rng, length=6, phys_dim=2, bond_dim=4)\n",
        "b = random_mps(rng, length=6, phys_dim=2, bond_dim=4)\n",
        "\n",
        "ip_mps = mps_inner(a, b)\n",
        "da = tf.reshape(mps_to_dense(a), (-1,))\n",
        "db = tf.reshape(mps_to_dense(b), (-1,))\n",
        "ip_dense = tf.reduce_sum(da * db)\n",
        "\n",
        "rel = tf.abs(ip_mps - ip_dense) / (tf.abs(ip_dense) + tf.cast(1e-15, tf.float64))\n",
        "print(\"relative inner mismatch:\", float(rel.numpy()))\n"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.x"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}
